<a href="https://colab.research.google.com/github/mojtabaSefidi/AI-Challenge-PoliceVsThieves/blob/main/Event_Detection_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


### Import & Installation

In [2]:
from collections import Counter, OrderedDict
import operator
import pandas as pd
import numpy as np
import itertools
import random
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
import tensorflow
from tensorflow import keras
from keras.layers import Embedding
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import pickle
import time
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight

In [4]:
!pip install -q sentence_transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 5.5 MB 59.8 MB/s 
     |████████████████████████████████| 1.3 MB 40.7 MB/s 
     |████████████████████████████████| 182 kB 14.6 MB/s 
     |████████████████████████████████| 7.6 MB 11.7 MB/s 


In [5]:
!pip install -q hazm
from hazm import *
from hazm import stopwords_list
!gdown 1-DGbE-g78E979ig_D-jR_L0Gfp5KY0qC
!unzip resources.zip

     |████████████████████████████████| 316 kB 34.1 MB/s 
     |████████████████████████████████| 1.4 MB 54.4 MB/s 
     |████████████████████████████████| 233 kB 59.4 MB/s 
Downloading...
From: https://drive.google.com/uc?id=1-DGbE-g78E979ig_D-jR_L0Gfp5KY0qC
To: /content/resources.zip
100% 30.6M/30.6M [00:00<00:00, 35.8MB/s]
Archive:  resources.zip
  inflating: chunker.model           
  inflating: langModel.mco           
   creating: lib/
  inflating: lib/liblinear-1.8.jar   
  inflating: lib/libsvm.jar          
  inflating: lib/log4j.jar           
  inflating: malt.jar                
  inflating: postagger.model         


In [6]:
Pars_Bert_Model = SentenceTransformer('HooshvareLab/bert-base-parsbert-uncased',device='cuda')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Sub-Functions Needed

In [7]:
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_url(tweet):
    return re.sub(r"http\S+", " ", tweet)

def remove_usernames(tweet):
    return re.sub('@[^\s]+',' ',tweet)

def remove_English(tweet):
  return re.sub(r'\s*[A-Za-z]+\b', ' ' , tweet).rstrip()

def remove(text):
  text = text.replace('ھ','ه')
  text = text.replace('ئ','ی')
  text = text.replace('ؤ','و')
  text = text.replace('إ','ا')
  text = text.replace('أ','ا')
  text = text.replace('َ','')
  text = text.replace('ُ','')
  text = text.replace('ِ','')
  text = text.replace('ّ','')
  text = text.replace('ء','')
  text = text.replace('ـ','')
  text = text.replace('،',' ')
  text = text.replace('،',' ')
  text = re.sub('[^\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC\u06F0-\u06F9 ]', ' ', text)
  return re.sub("[^0-9\u0600-\u06FF]+", " ", text).strip()

def control_numeric(text):
  text = text
  pattern_integer = r'\d+'
  pattern_float = "\d+\.\d+"
  text = re.sub(pattern_float, ' ', text)
  text = re.sub(pattern_integer, ' ', text)
  return text

def remove_nan(dataset,column):
  dataset = dataset[dataset[column].notna()]
  return dataset

def clean_tweet(tweet):
  tweet = remove(tweet)
  tweet = remove_emoji(tweet)
  tweet = remove_usernames(tweet)
  tweet = remove_url(tweet)
  tweet = remove_English(tweet)
  tweet = control_numeric(tweet)
  return Normalizer().normalize(tweet)

def tweet_to_token(tweet):
    return word_tokenize(tweet)

def remove_StopWords(text):
  return Normalizer().normalize('  '.join([word for word in text.split() if word not in stop_words]))

def calculate_max_length(tweet_array):
  max_length = 0
  for tweet in tweet_array:
    if max_length < len(tweet):
      max_length = len(tweet)
  return max_length

def Text2KeyPhrase(text):
  return kw_model.extract_keywords(text, keyphrase_ngram_range=(4, 7), stop_words=None)

def plot_history(history):
    
    plt.figure(figsize=(8,5),linewidth = 7, edgecolor="whitesmoke")    
    n = len(history.history['accuracy'])
    
    plt.plot(np.arange(0,n)+1,history.history['accuracy'], color='orange',marker=".")
    plt.plot(np.arange(0,n)+1,history.history['loss'],'b',marker=".")
    
    # offset both validation curves
    plt.plot(np.arange(0,n)+ 1,history.history['val_accuracy'],'r')  
    plt.plot(np.arange(0,n)+ 1,history.history['val_loss'],'g')
    
    plt.legend(['Train Acc','Train Loss','Val Acc','Val Loss'])
    plt.grid(True)
    
    # set vertical limit to 1
    plt.gca().set_ylim(0,1)

    plt.xlabel("Number of Epochs")
    plt.ylabel("Value")
    plt.suptitle("Learning Curve", size=16, y=0.927)
    plt.show()

def token2lemma(token):
  return Lemmatizer().lemmatize(token).split('#')[0]

def get_index(tupl, pos):
  return [i for i, tupl in enumerate(tupl) if tupl[-1] == pos]

def norm_token_persian(sentence):
  return word_tokenize(Normalizer().normalize(sentence))

def persian_pos(tokens):
  tagger = POSTagger(model='postagger.model')
  removed_roles = ['POSTP','CONJ','PUNC','NUM']
  part_of_speech = tagger.tag(tokens)
  remove_indices = list()
  for i in range(len(part_of_speech)):
    if part_of_speech[i][-1] in removed_roles:
      remove_indices.append(i)
  part_of_speech = [i for j, i in enumerate(part_of_speech) if j not in remove_indices]
  return part_of_speech

def persian_lemma_stem(POS):
  noun_index = get_index(POS,'N')
  # verb_index = get_index(POS,'V')
  POS = [*map(list, POS)]
  for i in noun_index:
    POS[i][0] = Stemmer().stem(POS[i][0])
  # for j in verb_index:
  #   POS[j][0] = token2lemma(POS[j][0])
  lemma_stem = [i[0] for i in POS]
  return POS, lemma_stem, ' '.join([word for word in lemma_stem])

def count_chars(text):
  return len(text)

def count_words(text):
  return len(text.split())

def generate_class_weight(y_train):    
    class_weights = {}
    classes = np.unique(y_train)
    weights = compute_class_weight(class_weight = "balanced", classes= classes, y= y_train)
    
    for i in range(len(classes)):
        class_weights[classes[i]] = weights[i]
    
    return class_weights

def segment_df2index(df_test, segment_col='Topic'):
  topic_indices = {}
  for topic in df_test['Topic'].unique():
    topic_indices[topic] = list(df_test[df_test['Topic']==topic].index.values)
  return topic_indices

def filter_by_index(np_array, row_indices):
    colunm_indices = list(range(0, np_array.shape[-1]))
    return np_array[np.ix_(row_indices, colunm_indices)]



In [8]:
import seaborn as sn
sn.set()
from matplotlib.pyplot import figure
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve, roc_auc_score


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_confusion_matrix(confusion_matrix, title='', cmap ='Greens'):
    df = pd.DataFrame(confusion_matrix, range(len(confusion_matrix)), range(len(confusion_matrix)))
    plt.figure(figsize=(6,4))
    if title == '' :
        plt.title('Confusion Matrix')
    else:
        plt.title('Confusion Matrix' + ' ' + title)
    sn.set(font_scale=1) # for label size
    sn.heatmap(df, annot=True, annot_kws={"size": 12},fmt='.0f',cmap=cmap) # font size
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()


def plot_precision_recall_curve(actual_labels, prediction, title='', file_name=None):
    precision, recall, thresholds = precision_recall_curve(actual_labels, prediction)
    fig, ax = plt.subplots()
    ax.plot(recall, precision, color='purple')

    # add axis labels to plot
    if title == '':
        plt.title(title)
    else:    
        plt.title('Precision-Recall Curve')
    ax.set_ylabel('Precision')
    ax.set_xlabel('Recall')

    # display plot
    plt.show()
    if file_name is not None:
        plt.savefig(file_name)     

def plot_roc_curve(actual_labels, prediction, title='', file_name=None):
    fpr, tpr, _ = roc_curve(actual_labels, prediction)
    plt.plot(fpr, tpr, markeactual_labelsr='.', label='Boost')
    if title == '':
        plt.title(title)
    else:    
        plt.title('ROC Learning Curves')
    plt.xlabel('false positive rate')
    plt.ylabel('true positive rate')
    plt.show()
    
    if file_name is not None:
        plt.savefig(file_name) 
        
        
def plot_metrics(history):
    metrics = ['loss', 'prc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
    if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
        plt.ylim([0.8,1])
    else:
        plt.ylim([0,1])

    plt.legend()
    
def plot_auc_curve(actual_labels, prediction, model_name, title='', file_name = None):
    fpr, tpr, _ = roc_curve(actual_labels,  prediction)
    auc = roc_auc_score(actual_labels, prediction).round(4)
    plt.figure(figsize=(6,4))
    if title == '':
        plt.title(title)
    else:    
        plt.title('AUC Learning Curves')
    plt.plot(fpr,tpr, label='Model: '+ model_name + ", AUC=" + str(auc), color='red')
    plt.legend(loc=4)
    plt.show()
    if file_name is not None:
        plt.savefig(file_name)

## DataProcessor Class

In [27]:
stop_words = set(['زیرا','از','به','در','چه','پس','ولی','چون','یا','نیز','را','و', 'هم' , 'اگر' ,'تا' ,'اما' , 'که', 'با', 'اگرچه', 'آن', 'این', 'اکنون','همین','همان','چنانچه','آ‌نگاه','چنانچه','چنانچه','بر','لایک', 'برای','است','آیا'])


In [28]:
class DataProcessor:
  
  def __init__(self, 
               Labeled_Dataset,
               Pars_Bert_Model,
               ):
        self.data = Labeled_Dataset
        self.Pars_Bert_Model = Pars_Bert_Model
    
  def split_data(self, split_ratio=0.2, validation_spilt=None):
    X_train, X_test, y_train, y_test = train_test_split(self.data.drop(['Label'], axis=1), self.data['Label'], test_size=split_ratio,
                                                        shuffle=True, stratify=self.data[['Label','Topic']])
    if not validation_spilt is None:
      X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_spilt,
                                                        shuffle=True, stratify=pd.concat([X_train['Topic'],y_train],axis=1))
      y_train = np.asarray(y_train).astype('int')
      y_test = np.asarray(y_test).astype('int')
      return X_train, X_test, X_validation, y_train, y_test, y_validation
    
    else:
      y_train = np.asarray(y_train).astype('int')
      y_test = np.asarray(y_test).astype('int')
      return X_train, X_test, y_train, y_test

  def column_selector(self, X_train, X_test, X_validation=None, feature_columns=['text']):
    if not X_validation is None:
      return X_train[feature_columns], X_test[feature_columns], X_validation[feature_columns]
    else:
      return X_train[feature_columns], X_test[feature_columns],

  def sampeler(self, X_train, y_train, strategy='Class_weight'):
    if strategy=='Over':
      sampler = RandomOverSampler(random_state=0)
    elif strategy=='Under':
      sampler = RandomUnderSampler(random_state=0)
    elif strategy=='Class_weight':
      return generate_class_weight(y_train)
    
    return sampler.fit_resample(X_train, y_train)

  def text2sequence(self, train_text, test_text, validation_text=None):
    print('1. Fit Tokenizer on Train...')
    tokenizer = text.Tokenizer()
    tokenizer.fit_on_texts(list(train_text))
    
    print('2. Convert Train Text to Sequence...')
    X_train_sequence = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train_text),
                                                                  maxlen=calculate_max_length(train_text),
                                                                  padding='post', truncating='post')
    
    print('3. Convert Test Text to Sequence...')
    X_test_sequence = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test_text),
                                                                 maxlen=calculate_max_length(train_text),
                                                                 padding='post', truncating='post')
    
    if not validation_text is None:
      print('4. Convert Validation Text to Sequence...')
      X_validation_sequence = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(validation_text),
                                                                         maxlen=calculate_max_length(train_text),
                                                                         padding='post', truncating='post')
      return X_train_sequence, X_test_sequence, X_validation_sequence
    
    else:
       return X_train_sequence, X_test_sequence
    
  def Bert_Embedding(self, train_text):
    print('1. Extract Tarin Embedding...')
    train_embedding = self.Pars_Bert_Model.encode(train_text)
    return train_embedding
    
    # print('Extract Test Embedding...')
    # test_embedding = self.Pars_Bert_Model.encode(test_text)
    
    # if not validation_text is None:
    #   print('Extract Validation Embedding...')
    #   validation_embedding = self.Pars_Bert_Model.encode(validation_text)
    #   return train_embedding, test_embedding, validation_embedding
    
    # else:
    #   return train_embedding, test_embedding
    
  def feature_extractor(self, X_train, X_test, X_validation=None):
    
    print('1. Remove StopWords from Train...')
    train_text = list(map(remove_StopWords, X_train['text']))
    print('2. Remove StopWords from Test...')
    test_text = list(map(remove_StopWords, X_test['text']))
    
    if not X_validation is None:
      print('3. Remove StopWords from Validation...')
      validation_text = list(map(remove_StopWords, X_validation['text']))

    print('text2sequence() Called...')
    if not X_validation is None:
      X_train_sequence, X_test_sequence, X_validation_sequence = self.text2sequence(train_text, test_text, validation_text)
    else:
      X_train_sequence, X_test_sequence = self.text2sequence(train_text, test_text)
    
    print('Bert_Embedding() Called...')
    X_train_embedding= self.Bert_Embedding(train_text)
    
    if not X_validation is None:
      return  X_train_sequence, X_train_embedding, X_test_sequence, X_validation_sequence
    else:
      return X_train_sequence, X_train_embedding, X_test_sequence,


  def Main(self):
    print('split_data() Called...')
    X_train, X_test, X_validation, y_train, y_test, y_validation = self.split_data(validation_spilt=0.15)
    topic_indices_test = segment_df2index(X_test, segment_col='Topic')
    print('column_selector() Called...')
    X_train, X_test, X_validation = self.column_selector(X_train, X_test, X_validation)
    print('sampeler() Called...')
    class_weight = self.sampeler(X_train, y_train)
    print('feature_extractor() Called...')
    X_train_sequence, X_train_embedding, X_test_sequence, X_validation_sequence = self.feature_extractor(X_train, X_test, X_validation)
    return X_train_sequence, X_train_embedding, y_train, X_test_sequence, y_test, X_validation_sequence, y_validation, class_weight, topic_indices_test

## Model Class

In [29]:
METRICS = [
      keras.metrics.TruePositives(name='Tp'),
      keras.metrics.FalsePositives(name='Fp'),
      keras.metrics.TrueNegatives(name='TN'),
      keras.metrics.FalseNegatives(name='FN'), 
      keras.metrics.BinaryAccuracy(name='Accuracy'),
      keras.metrics.Precision(name='Precision'),
      keras.metrics.Recall(name='Recall'),
      keras.metrics.AUC(name='AUC'),
      keras.metrics.AUC(name='PRC', curve='PR')
]

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_PRC', 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)

In [30]:
class Model:

  def __init__(self, 
               X_train,
               y_train,
               train_embedding,
               X_test,
               y_test,
               X_validaiton=None,
               y_validaiton=None
               ):
    
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.train_embedding = train_embedding
        if not y_validaiton is None:
          self.X_validaiton = X_validaiton
          self.y_validaiton = y_validaiton
  
  def define_cnn_model(self,               
                       filters = 250,               
                       kernel_size = 3,
                       units = 250,
                       drop_rate = 0.4,
                       learning_rate = 0.02):
      
      model = Sequential()
      
      #create embedding layer
      model.add(Embedding(self.train_embedding.shape[0], self.train_embedding.shape[1], trainable = False
                        ,embeddings_initializer = keras.initializers.Constant(self.train_embedding)))    
      # 1st dropout
      model.add(Dropout(drop_rate))
      
      # 1st convolutional 1-D layer
      model.add(Conv1D(filters, kernel_size, padding = 'valid')) #no padding
      
      #max pooling layer
      model.add(MaxPooling1D())

      # 2nd convolutional 1-D layer
      model.add(Conv1D(filters, kernel_size, padding = 'valid')) #no padding
      
      #max pooling layer
      model.add(MaxPooling1D())
      
      # 3rd convolutional 1-D layer
      model.add(Conv1D(filters, kernel_size, padding = 'valid', activation = 'relu'))
      
      # global max pooling layer
      model.add(GlobalAveragePooling1D())
      
      # 1st dense layer
      model.add(Dense(units, activation = 'relu'))
      
      # 2nd dropout
      model.add(Dropout(drop_rate))
      
      # final dense layer
      model.add(Dense(1,activation = 'sigmoid'))
      
      # compile the model
      model.compile(loss = 'binary_crossentropy',    #since we are doing binary classification
                  optimizer = tensorflow.optimizers.SGD(learning_rate = learning_rate),
                  metrics = METRICS)

      self.cnn_model = model
      
      return model.summary()
    
  def train_cnn(self, batch_size=128, epochs=100, class_weight=None):

    if not class_weight is None:
        cnn_history = self.cnn_model.fit(
            self.X_train, self.y_train,
            batch_size = batch_size,
            validation_data = [self.X_validaiton, self.y_validaiton],
            callbacks = [early_stopping],
            epochs = epochs,
            class_weight = class_weight,
            shuffle = True)
    else:
        cnn_history = self.cnn_model.fit(
            self.X_train, self.y_train,
            batch_size = batch_size,
            validation_data = [self.X_validaiton, self.y_validaiton],
            callbacks = [early_stopping],
            epochs = epochs,
            shuffle = True)
    
    return cnn_history
    
  def plot_cnn_training_history(self, history):
    plot_metrics(history)
    return plot_history(history)

  def Evaluate_cnn(self, batch_size=128, threshold=0.3, title=''):
    cnn_prediction = self.cnn_model.predict(self.X_test, verbose=1, batch_size=batch_size)
    cnn_prediction[cnn_prediction >= threshold] = 1
    cnn_prediction[cnn_prediction < threshold] = 0
    plot_confusion_matrix(confusion_matrix(self.y_test, cnn_prediction), title=title, cmap ='Greens')
    print(classification_report(self.y_test, cnn_prediction))
    plot_roc_curve(self.y_test, cnn_prediction, title='ROC Curve Of CNN Model', file_name=None)
    plot_auc_curve(self.y_test, cnn_prediction, model_name='CNN', title='AUC Curve Of CNN Model', file_name = None)
    plot_precision_recall_curve(cnn_prediction, cnn_prediction, title='Percision_Recall Curve Of CNN Model', file_name=None)
  
  def topic_base_evaluation(self, topic_indices, batch_size=128, threshold=0.3, title=''):
    
    prediction = self.cnn_model.predict(np_features, verbose=1, batch_size=batch_size)
    prediction[prediction >= threshold] = 1
    prediction[prediction < threshold] = 0
    plot_confusion_matrix(confusion_matrix(self.y_test, cnn_prediction), title=title, cmap ='Greens')
    print(classification_report(self.y_test, cnn_prediction))
    plot_roc_curve(self.y_test, cnn_prediction, title='ROC Curve Of CNN Model', file_name=None)
    plot_auc_curve(self.y_test, cnn_prediction, model_name='CNN', title='AUC Curve Of CNN Model', file_name = None)
    plot_precision_recall_curve(cnn_prediction, cnn_prediction, title='Percision_Recall Curve Of CNN Model', file_name=None)
    
      
  def Main(self):
    self.define_cnn_model()
    print()
    print('----------- Training of CNN Model Starts -----------')
    print()    
    self.train_cnn()
    self.plot_cnn_training_history()
    print()
    print('----------- Evaluation of CNN Model Starts -----------')
    print()
    self.Evaluate_cnn(title='Evaluation')


## Predict Class

In [ ]:
class Predict:
  
  def __init__(self, 
               SVM_model_tech,
               CNN_model_tech,
               tech_train_text,
               SVM_model_sport,
               CNN_model_sport,
               sport_train_text
               ):
        self.SVM_model_tech = SVM_model_tech
        self.CNN_model_tech = CNN_model_tech
        self.SVM_model_sport = SVM_model_sport
        self.CNN_model_sport = CNN_model_sport
        self.sport_train_text = sport_train_text
        self.tech_train_text = tech_train_text
        self.model = SentenceTransformer('HooshvareLab/bert-base-parsbert-uncased',device='cuda')
  
  def feature_extrction(self, pp_text, category):
    text_embedding = self.model.encode(pp_text)
    tokenizer = text.Tokenizer()
    if category == 'sport':
      tokenizer.fit_on_texts(list(self.sport_train_text))
    elif category == 'tech':
      tokenizer.fit_on_texts(list(self.tech_train_text))
    text_input = tokenizer.texts_to_sequences([pp_text])
    encoded_text = sequence.pad_sequences(text_input, maxlen = len(pp_text))
    return text_embedding, encoded_text

  def Main(self):
    while True:
      sample = input('Please Enter your Text: ')
      if sample == 'Quit':
        break
      category = input('Enter Category of Text (sport/tech): ')
      
      start_time = time.time()
      sample = clean_tweet(sample)
      text_embedding, encoded_text = self.feature_extrction(sample,category)

      if category == 'sport':
        cnn_prediction = self.CNN_model_sport.predict(encoded_text, verbose = 1, batch_size = 128)
        svm_prediction = self.SVM_model_sport.predict(text_embedding.reshape(-1, 768))
        end_time = time.time()
        print('input text: ',clean_tweet(sample))
        print('CNN model output: ', cnn_prediction)
        print('SVM model output: ', svm_prediction)
        print('Time Duration:',(end_time - start_time),'Seconds')
      
      elif category == 'tech':
        cnn_prediction = self.CNN_model_tech.predict(encoded_text, verbose = 1, batch_size = 128)
        svm_prediction = self.SVM_model_tech.predict(text_embedding.reshape(-1, 768))
        end_time = time.time()
        print('input text: ',clean_tweet(sample))
        print('CNN model output: ', cnn_prediction)
        print('SVM model output: ', svm_prediction)
        print('Time Duration:',(end_time - start_time),'Seconds')

#Test

In [21]:
%%time
final_dataset = pd.read_csv('/content/gdrive/MyDrive/Event Detection Final Dataset/EventDetection_All_Dataset.csv')
final_dataset.head()

CPU times: user 9.9 s, sys: 966 ms, total: 10.9 s
Wall time: 16.2 s


,datetime,text,retweet_count,like_count,username,hashtags,Label,Topic
0,2007-02-23 15:55:14+00:00,در یک نگاه نامزدهای جایزه اسکار بهترین فیلم,0.0,0.0,bbcpersian,Without_Hashtag,0,Art
1,2007-02-23 16:20:23+00:00,در یک نگاه نامزدهای جایزه اسکار بهترین فیلم,0.0,0.0,bbcpersian,Without_Hashtag,0,Art
2,2007-02-23 18:00:23+00:00,آیاتحریم‌های خارجی می‌تواند بر صنعت نفت ایران ...,0.0,0.0,bbcpersian,Without_Hashtag,0,Industry
3,2007-02-24 01:05:13+00:00,صفحه و ژه اسکار,0.0,0.0,bbcpersian,Without_Hashtag,0,Art
4,2007-02-24 12:35:09+00:00,آیاتحریم‌های خارجی می‌تواند بر صنعت نفت ایران ...,0.0,0.0,bbcpersian,Without_Hashtag,0,Industry


In [ ]:
%%time
data_processor = DataProcessor(final_dataset,Pars_Bert_Model)
X_train_sequence, X_train_embedding, y_train, X_test_sequence, y_test, X_validation_sequence, y_validation, class_weight, topic_indices_test = data_processor.Main()


split_data() Called...
column_selector() Called...
sampeler() Called...
feature_extractor() Called...
1. Remove StopWords from Train...
2. Remove StopWords from Test...
3. Remove StopWords from Validation...
text2sequence() Called...
Fit Tokenizer on Train...
1. Convert Train Text to Sequence...
2. Convert Test Text to Sequence...
3. Convert Validation Text to Sequence...
Bert_Embedding() Called...
1. Extract Tarin Embedding...


In [23]:
X_train_sequence.shape, X_train_embedding.shape, y_train.shape, X_test_sequence.shape, y_test.shape, X_validation_sequence.shape, y_validation.shape

NameError: ignored

In [ ]:
class_weight

In [ ]:
%%time
cnn_model = Model(
    X_train = X_train_sequence,
    y_train = y_train,
    train_embedding = X_train_embedding,
    X_test = X_test_sequence,
    y_test = y_test,
    X_validaiton = X_validation_sequence,
    y_validation =y_validation
               )

print('1. Define The Model...')
cnn_model.define_cnn_model()
print('2. Train The Model...')
cnn_model.train_cnn(class_weight=class_weight)
print('3. Plot Learning Process...')
cnn_model.plot_cnn_training_history()
print('4. Evaluate The Model...')
cnn_model.Evaluate_cnn(title='Evaluation')


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 768)         260957184 
                                                                 
 dropout_2 (Dropout)         (None, None, 768)         0         
                                                                 
 conv1d_3 (Conv1D)           (None, None, 250)         576250    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, None, 250)        0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, None, 250)         187750    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, None, 250)        0         
 1D)                                                  

# Save & Load models

In [ ]:
import pickle
pickle.dump(economy_event_detection.svm_model, open('Economy_svm_model', 'wb'))
# pickle.dump(Art_event_detection.svm_model, open('Art_svm_model', 'wb'))

# model_json = economy_event_detection.cnn_model.to_json()
# with open("Economy_cnn_model.json", "w") as json_file:
#     json_file.write(model_json)
# economy_event_detection.cnn_model.save_weights("Economy_cnn_model.h5")

model_json = Art_event_detection.cnn_model.to_json()
with open("Art_cnn_model.json", "w") as json_file:
    json_file.write(model_json)
Art_event_detection.cnn_model.save_weights("Art_cnn_model.h5")



import shutil
shutil.move("/content/Economy_svm_model", "/content/gdrive/MyDrive/Event_Detection_Models")
shutil.move("/content/Art_svm_model", "/content/gdrive/MyDrive/Event_Detection_Models")
shutil.move("/content/Economy_cnn_model.json", "/content/gdrive/MyDrive/Event_Detection_Models")
shutil.move("/content/Economy_cnn_model.h5", "/content/gdrive/MyDrive/Event_Detection_Models")
shutil.move("/content/Art_cnn_model.json", "/content/gdrive/MyDrive/Event_Detection_Models")
shutil.move("/content/Art_cnn_model.h5", "/content/gdrive/MyDrive/Event_Detection_Models")

NameError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json

json_file = open('/content/gdrive/MyDrive/Event_Detection_Models/Economy_cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/gdrive/MyDrive/Event_Detection_Models/Economy_cnn_model.h5")
print("Loaded model from disk")

## Real-Time Simulation

In [ ]:
Real_time_Prediction = Predict(tech_event_detection.svm_model,
                           tech_event_detection.cnn_model,
                           tech_processed.train_text,
                           sport_event_detection.svm_model,
                           sport_event_detection.cnn_model,
                           sport_processed.train_text)

In [ ]:
Real_time_Prediction.Main()

In [ ]:
def generate_class_weight(y_train):
    class_weights = {}
    classes = np.unique(y_train)
    weights = compute_class_weight(class_weight = "balanced", classes= classes, y= y_train)
    for i in range(len(classes)):
        class_weights[classes[i]] = weights[i]
    return class_weights